In [1]:
import sympy as sp
import torch

In [9]:
from src.v4.torchengine import AnalyticalSetSympy, EliminateAnalysis, ParallelResiduals
from src.v4.torchdata import symbols

In [3]:
x,y1,y2,y3,y4,u1,u2, indices = symbols('x y1 y2 y3 y4 u1 u2', dim='scalar')
analyses = {
    1: (y1, x),
    2: (u1, x+2-y3),
    3: (y2, x**2+u1),
    4: (u2, x**2-y1+y2),
    5: (y3, x-u2),
    6: (y4, x)
}

In [4]:
S = {idx: AnalyticalSetSympy(expr, outputvar=leftvar, indices=indices) for idx, (leftvar, expr) in analyses.items()}

In [5]:
S1 = S[1].analysis
S2 = EliminateAnalysis([S[2].analysis, S[3].analysis],[])
S3 = EliminateAnalysis([S[4].analysis, S[5].analysis],[])
S4 = S[6].analysis

In [6]:
sequential = EliminateAnalysis([S1, S2, S3, S4], [])

In [7]:
couplingvars = [elt for elt in sequential.structure[0] if elt in sequential.structure[1]]
couplingvars

[tensor(3)]

In [10]:
residuals = ParallelResiduals([sequential],[], sharedvars=couplingvars)

In [11]:
x0 = torch.tensor([1,2,3,0,5,0,0], dtype=torch.float64)

In [12]:
sequential(x0)

tensor([ 1.,  1.,  4., -3.,  1.,  3.,  4.], dtype=torch.float64)

In [13]:
residuals(x0)

tensor([-3.], dtype=torch.float64)

In [14]:
sequential.structure[0]

tensor([0, 3])